In [58]:
import jieba
import re
import sys

In [59]:
class Segmenter:
    def __init__(self, dictionary_path):
        self.dictionary = set()
        with open(dictionary_path, 'r', encoding='utf-8') as f:
            for line in f:
                line_list = line.split(' ')
                self.dictionary.add(line_list[0].strip())

    
    # 正向最大匹配算法
    def fmm(self, text):
        words = []
        ptr = 0
        max_length = max([len(w) for w in self.dictionary])
        while ptr < len(text):
            matched = False
            for i in range(max_length, 0, -1):
                if ptr + i > len(text):
                    continue
                word = text[ptr:ptr+i]
                if word in self.dictionary:
                    words.append(word)
                    ptr += i
                    matched = True
                    break
            if not matched:
                words.append(text[ptr])
                ptr += 1
        return words

    # 逆向最大匹配法
    def bmm(self, text):
        words = []
        ptr = len(text)
        max_length = max([len(w) for w in self.dictionary])
        while ptr > 0:
            matched = False
            for i in range(max_length, 0, -1):
                if ptr - i < 0:
                    continue
                word = text[ptr-i:ptr]
                if word in self.dictionary:
                    words.append(word)
                    ptr -= i
                    matched = True
                    break
            if not matched:
                words.append(text[ptr-1])
                ptr -= 1
        words.reverse()
        return words


if __name__ == '__main__':
    
    punc = "[,.:\"，。：；\'“”、 《》？?*!();_%\n\\-…—/！—@#￥%（）~·$`~‘’\0<>\t【】qwertyuiopasdfghjklzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNM]"
    segmenter = Segmenter(r'dict.txt')
    f = open(r'群鸦的盛宴.txt', 'r',encoding='utf-8')
    text = f.read()
    text = re.sub(punc, "", text)
    
    fmm_res = segmenter.fmm(text)
    i = 0
    out = open(r'fmm_res.txt', 'w',encoding='utf-8')
    for item in fmm_res:
        i += 1
        out.write(item + " / ")
        if i % 20 == 0:
            out.write("\n")
    
    bmm_res = segmenter.bmm(text)
    i = 0
    out = open(r'bmm_res.txt', 'w',encoding='utf-8')
    for item in bmm_res:
        i += 1
        out.write(item + " / ")
        if i % 20 == 0:
            out.write("\n")

In [60]:
print(bmm_res)

['冰', '与', '火', '之', '歌', '卷', '四', '群鸦', '的', '盛宴', '作者', '乔治', '马丁', '上', '序章', '\u3000', '\u3000', '龙', '莫', '兰德', '边', '说', '边', '从', '地上', '抓起', '一只', '干瘪', '的', '苹', '果在', '双手', '之间', '丢', '来', '丢去', '\u3000', '\u3000', '扔', '啊', '外号', '斯芬克斯', '的', '拉', '蕾', '萨', '催促', '他', '从', '箭', '囊', '里', '抽出', '一', '支箭', '搭上', '弓弦', '\u3000', '\u3000', '我', '想', '看', '龙', '鲁', '尼', '在', '他们', '当中', '年纪', '最小', '又', '矮', '又', '胖', '尚有', '两岁', '才', '成年', '哪怕', '一眼', '都', '好', '\u3000', '\u3000', '我', '想', '萝', '希', '搂', '着', '我', '睡觉', '佩特', '心想', '他', '坐在', '板凳', '上', '不安', '地', '挪', '动到', '明天', '早上', '女孩', '就是', '他', '的', '人', '了', '我', '要', '带', '她', '远离', '旧镇', '穿越', '狭', '海', '去', '自由贸易', '城邦', '那里', '没有', '学士', '没', '有人', '会', '抓', '他', '\u3000', '\u3000', '艾', '玛', '的', '笑声', '从', '头顶', '的', '窄', '窗', '中', '传出', '夹杂着', '恩', '客', '低沉', '的', '嗓门', '她', '乃', '羽', '笔', '酒', '樽', '最', '年长', '的', '女招待', '年', '过', '四十', '却是', '体态', '丰盈', '风韵犹存', '萝', '希', '是', '她', '女儿', '芳龄', '十五', '刚刚', '有'

In [61]:
def out(seg_list, filename):
    out = open(filename, 'w')
    for item in seg_list:
        out.write(item + '/')
    out.close()

with open(r'群鸦的盛宴.txt', 'r',encoding='utf-8') as f:
    text = f.read()
    punc = "[,.:\"，。：；\'“”、 《》？?*!();_%\n\\-…—/！—@#￥%（）~·$`~‘’\0<>\t【】qwertyuiopasdfghjklzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNM]"
    text = re.sub(punc, "", text)
    jieba.load_userdict(r'dict.txt')
    jieba_res = jieba.cut(text, cut_all=False, use_paddle=True)
    out(jieba_res, r'jieba_res.txt')

In [65]:
# 定义计算准确率、召回率和F1值的函数
def calculate_metrics(text):
    # 分别调用自己实现的分词函数和jieba分词函数
    my_result = segmenter.fmm(text)
    jieba_result = list(jieba.cut(text))
    # 统计相同的词语数量
    same_words = set(my_result) & set(jieba_result)
    same_words_count = len(same_words)
    # 计算总的词语数量
    my_total_words_count = len(my_result)
    jieba_total_words_count = len(jieba_result)
    # 计算准确率、召回率和F1值
    precision = same_words_count / my_total_words_count * 100
    recall = same_words_count / jieba_total_words_count * 100
    f1 = 2 * precision * recall / (precision + recall)
    # 返回结果
    return precision, recall, f1

# 测试示例
test_sentences = [
    '他的行为让我很不满意。',
    '中华人民共和国国旗由五星红旗组成。',
    '对于新手来说，学习编程并不容易。',
    '北京市是中国的首都。',
    '虽然下雨了，但是我还是要去上课。',
    '这本书的作者是美国作家约翰·格林。',
    '我喜欢看电影，尤其是动作片。',
    '在大街上行走时，我们应该注意交通安全。',
    '世界上有很多不同种类的动物。',
    '这个问题很难回答。',
    '他的话说得非常有道理。',
    '她的笑容很迷人。',
    '如果你遇到困难，不要轻易放弃。',
    '这首歌的歌词很有深意。',
    '我很喜欢这个城市的氛围。',
    '我觉得这个方案很不错。',
    '他在公园里散步。',
    '我们应该关心身体健康。',
    '我的梦想是成为一名科学家。',
    '他的英语说得很流利。',
    '这种手机的品质很好。',
    '人生苦短，我用Python。',
    '这个问题的答案不太好想。',
    '他是一个非常有才华的音乐家。',
    '这个演员的演技非常出色。',
    '这个城市的夜景很美。',
    '这家餐厅的菜品非常美味。',
    '人类已经登上了月球。',
    '我喜欢看小说。',
    '这个项目需要更多的人力资源。',
    '他总是能够在关键时刻发挥出自己的实力。',
    '这个公司的业务正在逐步扩张。',
    '这个地方的风景非常优美。',
    '这个城市的交通很方便。',
    '她是一个非常有耐心的人。',
    '这个节目的收视率很高。',
    '他非常擅长计算机编程。',
    '这个项目的进展情况很顺利。',
    '这个城市的居民非常友好。',
    '这个产品的设计很新颖。',
    '他在演讲中强调了环境保护的重要性。',
    '这个公司的股票价格正在上涨。',
    '这个小说讲述了一个感人的故事。',
    '这个问题的解决方案还需要进一步的研究。',
    '他在比赛中获得了第一名。',
    '这个产品的功能非常强大。',
    '这个城市的气候很宜人。',
    '这个节目的评价非常高。',
    '这个项目需要一个很强的团队来完成。',
    '他是一个非常有爱心的人。',
    '这个公司的业绩正在逐步提升。',
    '这个小说的情节非常引人入胜。',
    '这个问题的复杂度比较高。',
    '他在比赛中打破了世界纪录。',
    '这个产品的销量非常惊人。',
    '这个城市的治安非常好。',
    '这个节目的娱乐性很强。',
    '这个项目的目标非常明确。',
    '他是一个非常有责任感的人。',
    '这个公司的发展前景非常光明。',
    '这个小说的结局非常出人意料。',
    '这个问题的解决需要一些专业知识。',
    '他在比赛中表现得非常出色。',
    '这个产品的价格比较实惠。',
    '这个城市的建筑风格非常独特。',
    '这个节目的观众反响非常热烈。',
    '这个项目的进展速度非常快。',
    '他是一个非常有毅力的人。']

with open(r'50句分词结果_fmm.txt', 'w',encoding='utf-8') as f:
    for sentence in test_sentences:
        fmm_res = segmenter.fmm(sentence)
        print(fmm_res,file=f)
with open(r'50句分词结果_bmm.txt', 'w',encoding='utf-8') as f:
    for sentence in test_sentences:
        fmm_res = segmenter.bmm(sentence)
        print(fmm_res,file=f)
for sentence in test_sentences:
    precision, recall, f1 = calculate_metrics(sentence)
    print('句子：{}'.format(sentence))
    print('准确率：{:.2f}%'.format(precision))
    print('召回率：{:.2f}%'.format(recall))
    print('F1值：{:.2f}%'.format(f1))
    print('-' * 30)

句子：他的行为让我很不满意。
准确率：77.78%
召回率：77.78%
F1值：77.78%
------------------------------
句子：中华人民共和国国旗由五星红旗组成。
准确率：100.00%
召回率：100.00%
F1值：100.00%
------------------------------
句子：对于新手来说，学习编程并不容易。
准确率：80.00%
召回率：80.00%
F1值：80.00%
------------------------------
句子：北京市是中国的首都。
准确率：100.00%
召回率：100.00%
F1值：100.00%
------------------------------
句子：虽然下雨了，但是我还是要去上课。
准确率：100.00%
召回率：100.00%
F1值：100.00%
------------------------------
句子：这本书的作者是美国作家约翰·格林。
准确率：80.00%
召回率：80.00%
F1值：80.00%
------------------------------
句子：我喜欢看电影，尤其是动作片。
准确率：100.00%
召回率：100.00%
F1值：100.00%
------------------------------
句子：在大街上行走时，我们应该注意交通安全。
准确率：100.00%
召回率：100.00%
F1值：100.00%
------------------------------
句子：世界上有很多不同种类的动物。
准确率：87.50%
召回率：77.78%
F1值：82.35%
------------------------------
句子：这个问题很难回答。
准确率：66.67%
召回率：80.00%
F1值：72.73%
------------------------------
句子：他的话说得非常有道理。
准确率：100.00%
召回率：100.00%
F1值：100.00%
------------------------------
句子：她的笑容很迷人。
准确率：100.00%
召回率：100.00%
F1值：100.00%
------------------------------
句